In [1]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = torch.device('cpu')

In [2]:
import cv2

from yolort.utils.image_utils import letterbox, non_max_suppression
from yolort.utils import (
    cv2_imshow,
    get_image_from_url,
    read_image_to_tensor,
    update_module_state_from_ultralytics,
)

## Prepare image and model weights to test

In [3]:
is_half = False

# img_path = "https://raw.githubusercontent.com/zhiqwang/yolov5-rt-stack/master/test/assets/bus.jpg"
img_path = "https://gitee.com/zhiqwang/yolov5-rt-stack/raw/master/test/assets/bus.jpg"  # For user in China
img_raw = get_image_from_url(img_path)

yolort_weight_path = '../weights/yolov5s_r40_updated.pt'
ultralytics_weights_path = "../weights/yolov5s.pt"

You can download the weight with following methods

```python
torch.hub.download_url_to_file(
    r'https://github.com/ultralytics/yolov5/releases/download/v4.0/yolov5s.pt',
    ultralytics_weights_path,
    progress=True,
)
```

In [4]:
# Preprocess
img = letterbox(img_raw, new_shape=(640, 640))[0]
img = read_image_to_tensor(img, is_half)
img = img.to(device)

## Load model as ultralytics and inference

In [18]:
conf = 0.25
iou = 0.45

model = torch.hub.load('ultralytics/yolov5', 'custom', path=ultralytics_weights_path,autoshape=False, force_reload=True)
model = model.to(device)
model.conf = conf  # confidence threshold (0-1)
model.iou = iou  # NMS IoU threshold (0-1)
model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
model.eval()

# Get actual anchors from ultralytics model
m = model.model[-1]  # get Detect() layer
anchor_grids = m.anchor_grid.squeeze().view((3,6)).tolist()  # get anchors

with torch.no_grad():
    ultralytics_dets = model(img[None])[0]
    ultralytics_dets = non_max_suppression(ultralytics_dets, conf, iou, agnostic=True)[0]

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /media/data03/zal-td-auto/mgromniak/.cache/torch/hub/master.zip
Fusing layers... 
Model Summary: 224 layers, 7078183 parameters, 0 gradients
YOLOv5 🚀 2021-6-16 torch 1.8.1+cu102 CUDA:0 (Tesla V100-SXM2-32GB, 32510.5MB)



In [19]:
print(f'Detection results with ultralytics:\n{ultralytics_dets}')

Detection results with ultralytics:
tensor([[157.49896, 258.56213, 183.10294, 266.86627,   0.83721,   0.00000],
        [ 66.84713, 255.05643,  93.06948, 262.18393,   0.75076,   0.00000]])


## Updating model weights from ultralytics to yolort

In [13]:
model = update_module_state_from_ultralytics(arch='yolov5s',
                                             version='v4.0',
                                             custom_path_or_model=ultralytics_weights_path,
                                             set_fp16=is_half,
                                             num_classes=80)

torch.save(model.state_dict(), yolort_weight_path)

Using cache found in /media/data03/zal-td-auto/mgromniak/.cache/torch/hub/ultralytics_yolov5_v4.0

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.co

RuntimeError: The size of tensor a (255) must match the size of tensor b (45) at non-singleton dimension 0

## Load model as yolort and inference

In [14]:
from yolort.models.yolo import yolov5_darknet_pan_s_r40 as yolov5s

model = yolov5s(score_thresh=conf, nms_thresh=iou, num_classes=80,anchor_grids=anchor_grids)
model.load_state_dict(torch.load(yolort_weight_path))

# Load model
if is_half:
    model.half()  # to FP16
model.eval()

with torch.no_grad():
    yolort_dets = model(img[None])

torch.Size([1, 3, 640, 480])


In [15]:
print(f"Detection boxes with yolort:\n{yolort_dets[0]['boxes']}")

Detection boxes with yolort:
tensor([[419.11774, 314.59589, 456.34717, 439.40979],
        [152.50375, 318.81580, 184.49927, 432.12903],
        [ 63.68687, 324.12653, 111.35161, 447.21997],
        [143.69586, 232.37770, 348.68765, 352.55768],
        [  6.62933, 363.97397,  37.73249, 458.05673]])


In [16]:
print(f"Detection scores with yolort:\n{yolort_dets[0]['scores']}")

Detection scores with yolort:
tensor([0.88143, 0.87442, 0.85041, 0.71293, 0.40053])


In [17]:
print(f"Detection labels with yolort:\n{yolort_dets[0]['labels']}")

Detection labels with yolort:
tensor([0, 0, 0, 5, 0])


## Varify the detection results between yolort and ultralytics

In [19]:
torch.testing.assert_allclose(yolort_dets[0]['boxes'], ultralytics_dets[:, :4], rtol=1e-05, atol=1e-05)

print("Exported model has been tested, and the result looks good!")

AssertionError: expected tensor shape torch.Size([2, 4]) doesn't match with actual tensor shape torch.Size([5, 4])!